In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import visvalingamwyatt as vw
import yfinance as yf

In [2]:
# Отрисовка свечного графика
def candles(ax, data, interval, alpha=1):
    coeff = {"1m":1/24/60, "2m":1/24/30, "5m":1/24/12, "15m":1/24/4, "30m":1/24/2,
        "60m":1/24, "90m":1/16, "1h":1/24, "1d":1, "5d":5, "1wk":7, "1mo":30,
         "3mo":90}
    up = data[data.Close >= data.Open]
    down = data[data.Close < data.Open]
    col_up = 'green'
    col_down = 'red'
    col_shadow_up = 'black'
    col_shadow_down = 'black'
    width_body = .8 * coeff[interval]
    width_shadow = .2 * coeff[interval]
    ax.bar(up.index, up.Close-up.Open, width_body, bottom=up.Open, color=col_up, alpha=alpha)
    ax.bar(up.index, up.High-up.Close, width_shadow, bottom=up.Close, color=col_shadow_up, alpha=alpha)
    ax.bar(up.index, up.Low-up.Open, width_shadow, bottom=up.Open, color=col_shadow_up, alpha=alpha)
    ax.bar(down.index, down.Close-down.Open, width_body, bottom=down.Open, color=col_down, alpha=alpha)
    ax.bar(down.index, down.High-down.Open, width_shadow, bottom=down.Open, color=col_shadow_down, alpha=alpha)
    ax.bar(down.index, down.Low-down.Close, width_shadow, bottom=down.Close, color=col_shadow_down, alpha=alpha)

In [3]:
def get_data(ticker, period, interval):
    return yf.download(ticker, period=period, interval=interval)
# Сглаживаем график и объединяем однонаправленные отрезки
def simplify(data, segments):
    temp = [[i, data["Close"][i]] for i in range(len(data))]
    temp = vw.simplify(temp, number=segments) 
    return data.iloc[[i[0] for i in temp]]
def simplify_monotone(data):
    temp = [0]
    for i in range(1, data.shape[0]-1):
        if (data["Close"][i+1]-data["Close"][i] > 0) != (data["Close"][i]-data["Close"][i-1] > 0):
            temp.append(i)
    temp.append(data.shape[0]-1)
    return data.iloc[temp]
# Ищем паттерн
def search_head_and_shoulders(data):
    i = 0
    if data["Close"][0] > data["Close"][1]:
        i += 1
    res = []
    while i <= data.shape[0]-7:
        left = data["Close"][i+1]
        left_min = data["Close"][i+2]
        head = data["Close"][i+3]
        right_min = data["Close"][i+4]
        right = data["Close"][i+5]
        if (head > left and head > right and head - max(left, right) > abs(left-right) 
            and min(left, right) - max(left_min, right_min) > abs(left-right)):
            res.append(data.iloc[i:i+7])
            i += 4
        i += 2
    
    # Обратный паттерн
    i = 0
    if data["Close"][0] < data["Close"][1]:
        i += 1
    while i <= data.shape[0]-7:
        left = data["Close"][i+1]
        left_min = data["Close"][i+2]
        head = data["Close"][i+3]
        right_min = data["Close"][i+4]
        right = data["Close"][i+5]
        if (head < left and head < right and min(left, right) - head > abs(left-right) 
            and min(left_min, right_min) - max(left, right) > abs(left-right)):
            res.append(data.iloc[i:i+7])
            i += 4
        i += 2
    return res

In [4]:
def res(ticker="BTC-USD", period="1y", interval="1d", segments=30):
    data = get_data(ticker, period, interval)
    
    fig = plt.figure(figsize=[12.8, 9.6])

    res1 = simplify(data, segments)
    res2 = simplify_monotone(res1)
    patterns = search_head_and_shoulders(res2)

    ax1 = fig.add_subplot(321)
    candles(ax1, data, interval)
    ax1.set_title("Свечной график")

    ax2 = fig.add_subplot(322)
    ax2.plot(data.index, data.Close)
    ax2.set_title("Цена закрытия")

    ax3 = fig.add_subplot(323)
    ax3.plot(res1.index, res1.Close)
    ax3.set_title("Сглаживание")

    ax4 = fig.add_subplot(324)
    ax4.plot(res2.index, res2.Close)
    ax4.set_title("Объединение монотонных отрезков")

    ax5 = fig.add_subplot(325)
    ax5.plot(res2.index, res2.Close)
    for pattern in patterns:
        ax5.plot(pattern.index, pattern.Close, lw=5, c="black")
    ax5.set_title("Поиск паттерна")

    ax6 = fig.add_subplot(326)
    candles(ax6, data, interval)
    for pattern in patterns:
        ax6.plot(pattern.index, pattern.Close, lw=5, c="black")
    ax6.set_title("Результат")

    fig.suptitle(ticker)
    fig.subplots_adjust(top=0.94, hspace=0.3)
    for ax in fig.axes:
        plt.setp(ax.get_xticklabels(), rotation=30)
    plt.show()
    
ticker_list = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD", 
               "XRP-USD", "ADA-USD", "DOGE-USD", "STETH-USD", "HEX-USD", 
               "MATIC-USD", "SOL-USD", "DOT-USD", "LTC-USD", "WTRX-USD"]
period_list = ["1h", "1d", "1wk", "1mo", "3mo", "6mo", "1y", "2y", "3y", "5y"]
interval_list = ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]

interactive_plot = widgets.interactive(res, {'manual': True}, ticker=ticker_list, period=period_list, interval=interval_list, 
                                       segments=(10, 80, 5))
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot

interactive(children=(Dropdown(description='ticker', options=('BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'US…